<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# Building a Large Language Model from Scratch — A Step-by-Step Guide Using Python and PyTorch
## Project Companion — attoLLM Colab Starter
**© Dr. Yves J. Hilpisch**<br>AI-Powered by GPT-5.

## How to Use This Notebook

- Sanity-check GPU, disk, and Python versions before diving into model training.
- Mount or sync the storage location you plan to use for datasets and checkpoints.
- Bookmark the helper utilities you expect to reuse across multiple chapters.

### Starter Checklist

Use the diagnostics cells here before every new Colab session. Consistent environments remove a whole class of hard-to-track bugs.

### Collaboration Notes

Share a copy of this notebook with teammates so everyone runs the same validation steps before kicking off workloads.

In [ ]:
!nvidia-smi || echo 'No NVIDIA GPU available'

In [ ]:
# Install a recent CUDA wheel when GPU is present; fallback to CPU wheel
import os
import subprocess
import sys


def ensure_torch():
    try:
        import torch  # type: ignore
        return torch
    except Exception as exc:  # pragma: no cover - diagnostics
        print('torch import failed:', exc)
        if os.environ.get('COLAB_RELEASE_TAG'):
            gpu_available = os.system('nvidia-smi > /dev/null 2>&1') == 0
            index = 'https://download.pytorch.org/whl/cu121' if gpu_available else 'https://download.pytorch.org/whl/cpu'
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'torch', '--index-url', index], check=True)
            import torch  # type: ignore
            return torch
        return None


torch = ensure_torch()
if torch is None:
    print('Install torch manually when you have network access (skipped for validation).')
else:
    print('torch:', torch.__version__, 'cuda?', torch.cuda.is_available())


In [ ]:
# Scaffold attoLLM in /content/attollm (Colab) or a local folder otherwise
import os
import pathlib
import subprocess
import sys
import textwrap

root = pathlib.Path('/content/attollm') if os.environ.get('COLAB_RELEASE_TAG') else pathlib.Path.cwd() / 'attollm_scaffold'

(root / 'src/attollm').mkdir(parents=True, exist_ok=True)
for d in ['scripts', 'configs', 'data/raw', 'data/processed', 'data/cache', 'checkpoints', 'tests']:
    (root / d).mkdir(parents=True, exist_ok=True)

(root / 'README.md').write_text('# attoLLM (Colab)\n')

gitignore_text = '\n'.join([
    '__pycache__/',
    '*.pyc',
    'data/cache/',
    'checkpoints/',
    'runs/',
    '*.pt',
    '*.pth',
]) + '\n'
(root / '.gitignore').write_text(gitignore_text)

requirements_text = 'numpy>=1.24\n' 'tqdm>=4.66\n'
(root / 'requirements.txt').write_text(requirements_text)

pyproject_text = textwrap.dedent("""
    [build-system]
    requires = ["setuptools>=68", "wheel"]
    build-backend = "setuptools.build_meta"

    [project]
    name = "attollm"
    version = "0.0.1"
    requires-python = ">=3.10"
    dependencies = []

    [tool.setuptools]
    package-dir = {"" = "src"}

    [tool.setuptools.packages.find]
    where = ["src"]
""")
(root / 'pyproject.toml').write_text(pyproject_text)

(root / 'src/attollm/__init__.py').write_text('__all__ = ["hello"]\n')

hello_py = textwrap.dedent("""
    def main():
        print("Hello from attoLLM (Colab)!")

    if __name__ == "__main__":
        main()
""")
(root / 'src/attollm/hello.py').write_text(hello_py)

print('Scaffolded at:', root)

if os.environ.get('COLAB_RELEASE_TAG'):
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', str(root)], check=True)
    subprocess.run([sys.executable, '-m', 'attollm.hello'], check=True)
else:
    print('Running outside Colab; editable install skipped to keep validation fast.')
    print('Inspect the scaffolded package locally at', root)

## Exercises

- Adapt the environment check to alert you when CUDA is missing or mismatched.
- Mount your preferred cloud storage (Drive, S3 via fsspec, etc.) and verify read/write access.
- Create a short markdown playbook describing how you spin up a fresh Colab runtime for this project.

<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>